In [1]:
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

model_name = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = TFBertModel.from_pretrained(model_name)
texts = ["這是一個很棒的產品！", "這家餐廳的服務很差...", "電影的劇情令人感動！"]
inputs = tokenizer(
    texts, return_tensors="tf", padding=True, truncation=True, max_length=512
)
outputs = bert_model(**inputs)
cls_embeddings = outputs.last_hidden_state[:, 0, :]
print("BERT 特徵維度:", cls_embeddings.shape)
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model

input_layer = Input(shape=(768,))  # BERT 特徵維度
x = Dense(128, activation="relu")(input_layer)
x = Dense(64, activation="relu")(x)
output_layer = Dense(2, activation="softmax")(x)  # 二分類問題

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

BERT 特徵維度: (3, 768)


In [2]:
classifier = Model(inputs=input_layer, outputs=output_layer)
classifier.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
classifier.summary()
X_train = np.random.rand(100, 768)  # 100 筆訓練樣本，每個 768 維
y_train = np.random.randint(0, 2, 100)  # 0（負面）或 1（正面）
classifier.fit(X_train, y_train, epochs=5, batch_size=8)
test_texts = ["這部電影真的很棒！", "這間餐廳讓我失望透頂...", "這款手機的性能很不錯"]
test_inputs = tokenizer(
    test_texts, return_tensors="tf", padding=True, truncation=True, max_length=512
)
test_outputs = bert_model(**test_inputs)
test_cls_embeddings = test_outputs.last_hidden_state[:, 0, :]  # 取 CLS 向量
predictions = classifier.predict(test_cls_embeddings)
predicted_labels = predictions.argmax(axis=1)  # 取最大機率的類別
print("預測結果:", predicted_labels)
# 假設 1 代表正向，0 代表負向

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 768)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          98,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 106,818 (417.26 KB)

 Trainable params: 106,818 (417.26 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5337 - loss: 0.7177
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6330 - loss: 0.6229
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6192 - loss: 0.6338 
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7096 - loss: 0.5469
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7583 - loss: 0.4564 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
預測結果: [0 1 0]


In [5]:
test_texts = ["這部電影不錯，但是有點無聊。", "這部電影有點無聊，但是不錯。"]
test_inputs = tokenizer(
    test_texts, return_tensors="tf", padding=True, truncation=True, max_length=512
)
test_outputs = bert_model(**test_inputs)
test_cls_embeddings = test_outputs.last_hidden_state[:, 0, :]
predictions = classifier.predict(test_cls_embeddings)
predicted_labels = predictions.argmax(axis=1)  # 取最大機率的類別
print("預測結果:", predicted_labels)
test_texts = ["這是一個非常非常非常棒的產品！"]
test_inputs = tokenizer(
    test_texts, return_tensors="tf", padding=True, truncation=True, max_length=512
)
test_outputs = bert_model(**test_inputs)
test_cls_embeddings = test_outputs.last_hidden_state[:, 0, :]
predictions = classifier.predict(test_cls_embeddings)
predicted_labels = predictions.argmax(axis=1)  # 取最大機率的類別
print("預測結果:", predicted_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
預測結果: [0 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
預測結果: [0]


In [3]:
test_texts = ["這是一個非常非常非常棒的產品！"]
test_inputs = tokenizer(
    test_texts, return_tensors="tf", padding=True, truncation=True, max_length=512
)
test_outputs = bert_model(**test_inputs)
test_cls_embeddings = test_outputs.last_hidden_state[:, 0, :]
predictions = classifier.predict(test_cls_embeddings)
predicted_labels = predictions.argmax(axis=1)  # 取最大機率的類別
print("預測結果:", predicted_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
預測結果: [0]


In [4]:
from tensorflow.keras.layers import LSTM

input_layer = Input(shape=(768, 1))  # 修改
x = LSTM(128, return_sequences=False)(input_layer)  # 修改  LSTM
x = Dense(64, activation="relu")(x)
output_layer = Dense(2, activation="softmax")(x)
lstm_classifier = Model(inputs=input_layer, outputs=output_layer)
lstm_classifier.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
X_train_lstm = X_train.reshape(-1, 768, 1)
lstm_classifier.fit(X_train_lstm, y_train, epochs=5, batch_size=8)

Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 258ms/step - accuracy: 0.4924 - loss: 0.6976
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 247ms/step - accuracy: 0.4798 - loss: 0.6933
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 263ms/step - accuracy: 0.4790 - loss: 0.6944
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 252ms/step - accuracy: 0.5009 - loss: 0.6939
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 251ms/step - accuracy: 0.5937 - loss: 0.6817


In [5]:
test_texts = ["這部電影真的很棒！", "這間餐廳讓我失望透頂...", "這款手機的性能很不錯"]
test_inputs = tokenizer(
    test_texts, return_tensors="tf", padding=True, truncation=True, max_length=512
)
test_outputs = bert_model(**test_inputs)
test_cls_embeddings = test_outputs.last_hidden_state[:, 0, :]
predictions = lstm_classifier.predict(test_cls_embeddings)
predicted_labels = predictions.argmax(axis=1)  # 取最大機率的類別
print("預測結果:", predicted_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step
預測結果: [1 0 1]


In [6]:
test_texts = ["這部電影不錯，但是有點無聊。", "這部電影有點無聊，但是不錯。"]
test_inputs = tokenizer(
    test_texts, return_tensors="tf", padding=True, truncation=True, max_length=512
)
test_outputs = bert_model(**test_inputs)
test_cls_embeddings = test_outputs.last_hidden_state[:, 0, :]
predictions = lstm_classifier.predict(test_cls_embeddings)
predicted_labels = predictions.argmax(axis=1)  # 取最大機率的類別
print("預測結果:", predicted_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 459ms/step
預測結果: [0 0]


In [7]:
test_texts = ["這是一個非常非常非常棒的產品！"]
test_inputs = tokenizer(
    test_texts, return_tensors="tf", padding=True, truncation=True, max_length=512
)
test_outputs = bert_model(**test_inputs)
test_cls_embeddings = test_outputs.last_hidden_state[:, 0, :]
predictions = lstm_classifier.predict(test_cls_embeddings)
predicted_labels = predictions.argmax(axis=1)  # 取最大機率的類別
print("預測結果:", predicted_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
預測結果: [0]


In [1]:
from tensorflow.keras.layers import LSTM, Dense, Input
from transformers import (  # LSTM之後的修改? 使用 BERT 隱層輸出（整個序列）作為 LSTM 輸入 。
    BertTokenizer,
    TFBertModel,
)

model_name = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = TFBertModel.from_pretrained(model_name)
texts = ["這是一個很棒的產品！", "這家餐廳的服務很差...", "電影的劇情令人感動！"]
inputs = tokenizer(
    texts, return_tensors="tf", padding=True, truncation=True, max_length=512
)
outputs = bert_model(**inputs)
sequence_output = outputs.last_hidden_state  # (batch_size, seq_length, hidden_dim)
input_layer = Input(shape=(512, 768))  # BERT 由50改為預設的512
x = LSTM(128, return_sequences=True)(input_layer)
x = LSTM(64)(x)
output_layer = Dense(2, activation="softmax")(x)  # 二分類
lstm_model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
lstm_model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
import numpy as np

X_train = np.random.rand(
    100, 512, 768
)  # 100 筆訓練樣本，每個 512 tokens，每個 token 768 維
y_train = np.random.randint(0, 2, 100)
lstm_model.fit(X_train, y_train, epochs=15, batch_size=8)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 556ms/step - accuracy: 0.5523 - loss: 0.7897
Epoch 2/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 559ms/step - accuracy: 0.5139 - loss: 0.6974
Epoch 3/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 527ms/step - accuracy: 0.4577 - loss: 0.7375
Epoch 4/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 547ms/step - accuracy: 0.6477 - loss: 0.6797
Epoch 5/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 574ms/step - accuracy: 0.5545 - loss: 0.6964
Epoch 6/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 513ms/step - accuracy: 0.5410 - loss: 0.6805
Epoch 7/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 531ms/step - accuracy: 0.5890 - loss: 0.6664
Epoch 8/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 512ms/step - accuracy: 0.6075 - loss: 0.6585
Epoch 9/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 550ms/step - accuracy: 0.5971 - loss: 0.6571
Epoch 10/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 555ms/step - accuracy: 0.5908 - loss: 0.6371
Epoch 11/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 531ms/step - accuracy: 0.6671 - loss: 0.6343
Epoch 12/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 546ms

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.models import Model

test_texts = ["這部電影真的很棒！", "這間餐廳讓我失望透頂...", "這款手機的性能很不錯"]
test_inputs = tokenizer(
    test_texts, return_tensors="tf", padding=True, truncation=True, max_length=512
)
test_outputs = bert_model(**test_inputs)
#
# inputs = tokenizer(texts, return_tensors="tf", padding=True, truncation=True, max_length=512)
# outputs = bert_model(**inputs)
#
# test_cls_embeddings = test_outputs.last_hidden_state[:, 0, :]
test_cls_embeddings = test_outputs.last_hidden_state
print(test_cls_embeddings.shape)
print(type(test_inputs))
print(test_inputs.keys())  # 確保包含 'input_ids' 和 'attention_mask'


# LSTM 輸入層
input_layer = Input(shape=(None, 768))  # `None` 允許變長輸入
# input_layer = Input(shape=(512, 768))  # 512 為 BERT 序列長度, 768 為 BERT hidden size
x = LSTM(128, return_sequences=True)(input_layer)
x = LSTM(64)(x)
output_layer = Dense(2, activation="softmax")(x)  # 二分類

lstm_model = Model(inputs=input_layer, outputs=output_layer)
lstm_model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# 假設訓練數據
X_train = np.random.rand(
    100, 512, 768
)  # 100 筆訓練樣本，每個 512 tokens，每個 token 768 維
y_train = np.random.randint(0, 2, 100)

# 訓練 LSTM
lstm_model.fit(X_train, y_train, epochs=15, batch_size=8)

# 測試時轉換為 NumPy
test_cls_embeddings_np = test_cls_embeddings.numpy()
predictions = lstm_model.predict(test_cls_embeddings_np)
predicted_labels = predictions.argmax(axis=1)  # 取最大機率的類別
print("預測結果:", predicted_labels)

(3, 15, 768)
<class 'transformers.tokenization_utils_base.BatchEncoding'>
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
Epoch 1/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 534ms/step - accuracy: 0.4635 - loss: 0.7107
Epoch 2/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 543ms/step - accuracy: 0.4769 - loss: 0.6937
Epoch 3/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 547ms/step - accuracy: 0.5572 - loss: 0.7090
Epoch 4/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 559ms/step - accuracy: 0.5709 - loss: 0.7007
Epoch 5/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 599ms/step - accuracy: 0.5592 - loss: 0.6754
Epoch 6/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 11s 622ms/step - accuracy: 0.6631 - loss: 0.6653
Epoch 7/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 614ms/step - accuracy: 0.6746 - loss: 0.6521
Epoch 8/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 594ms/step - accuracy: 0.5310 - loss: 0.6340
Epoch 9/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 635ms/step - accuracy: 0.7707 - loss: 0.5853
Epoch 10/15
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 634ms/step - accuracy: 0.8492 - loss: 0.5039
E

In [8]:
test_texts = ["這部電影不錯，但是有點無聊。", "這部電影有點無聊，但是不錯。"]
test_inputs = tokenizer(
    test_texts, return_tensors="tf", padding=True, truncation=True, max_length=512
)
test_outputs = bert_model(**test_inputs)
test_cls_embeddings = test_outputs.last_hidden_state
# 測試時轉換為 NumPy
test_cls_embeddings_np = test_cls_embeddings.numpy()
predictions = lstm_model.predict(test_cls_embeddings_np)
predicted_labels = predictions.argmax(axis=1)  # 取最大機率的類別
print("預測結果:", predicted_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 545ms/step
預測結果: [0 0]


In [9]:
test_texts = ["這是一個非常非常非常棒的產品！"]
test_inputs = tokenizer(
    test_texts, return_tensors="tf", padding=True, truncation=True, max_length=512
)
test_outputs = bert_model(**test_inputs)
test_cls_embeddings = test_outputs.last_hidden_state
# 測試時轉換為 NumPy
test_cls_embeddings_np = test_cls_embeddings.numpy()
predictions = lstm_model.predict(test_cls_embeddings_np)
predicted_labels = predictions.argmax(axis=1)  # 取最大機率的類別
print("預測結果:", predicted_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
預測結果: [0]


In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Input, Multiply
from tensorflow.keras.models import Model
from transformers import (  # LSTM之後的修改? 使用 BERT 隱層輸出（整個序列）作為 LSTM 輸入 。
    BertTokenizer,
    TFBertModel,
)

model_name = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = TFBertModel.from_pretrained(model_name)

# 假設訓練數據
X_train = np.random.rand(
    1000, 512, 768
)  # 100 筆訓練樣本，每個 512 tokens，每個 token 768 維
y_train = np.random.randint(0, 2, 1000)


class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(
            shape=(input_shape[-1],), initializer="random_normal", trainable=True
        )
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        score = tf.nn.softmax(tf.tensordot(inputs, self.W, axes=1), axis=1)  # 計算權重
        return Multiply()([inputs, tf.expand_dims(score, -1)])  # 權重應用到輸入


input_layer = Input(shape=(None, 768))

# x = LSTM(128, return_sequences=True)(input_layer)
# x = LSTM(64)(x)

x = LSTM(256, dropout=0.2, return_sequences=True)(input_layer)
x = AttentionLayer()(x)  # 應用注意力機制
x = LSTM(128, dropout=0.2, return_sequences=True)(x)
x = AttentionLayer()(x)  # 應用注意力機制
x = LSTM(64, dropout=0.2)(x)
output_layer = Dense(2, activation="softmax")(x)
lstm_attention_model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
lstm_attention_model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
lstm_attention_model.fit(X_train, y_train, epochs=15, batch_size=128)

test_texts = ["這部電影不錯，但是有點無聊。", "這部電影有點無聊，但是不錯。"]
test_inputs = tokenizer(
    test_texts, return_tensors="tf", padding=True, truncation=True, max_length=512
)
test_outputs = bert_model(**test_inputs)
# test_cls_embeddings = test_outputs.last_hidden_state[:, 0, :]
test_embeddings = test_outputs.last_hidden_state  # 保持形狀 (batch_size, 512, 768)

predictions = lstm_attention_model.predict(test_embeddings)
predicted_labels = predictions.argmax(axis=1)  # 取最大機率的類別
print("預測結果:", predicted_labels)

test_texts = ["這是一個非常非常非常棒的產品！"]
test_inputs = tokenizer(
    test_texts, return_tensors="tf", padding=True, truncation=True, max_length=512
)
test_outputs = bert_model(**test_inputs)
test_cls_embeddings = test_outputs.last_hidden_state
# 測試時轉換為 NumPy
test_cls_embeddings_np = test_cls_embeddings.numpy()
predictions = lstm_attention_model.predict(test_cls_embeddings_np)
predicted_labels = predictions.argmax(axis=1)  # 取最大機率的類別
print("預測結果:", predicted_labels)


test_texts = ["這部電影真的很棒！", "這間餐廳讓我失望透頂...", "這款手機的性能很不錯"]
test_inputs = tokenizer(
    test_texts, return_tensors="tf", padding=True, truncation=True, max_length=512
)
test_outputs = bert_model(**test_inputs)
test_cls_embeddings = test_outputs.last_hidden_state
# 測試時轉換為 NumPy
test_cls_embeddings_np = test_cls_embeddings.numpy()
predictions = lstm_attention_model.predict(test_cls_embeddings_np)
predicted_labels = predictions.argmax(axis=1)  # 取最大機率的類別
print("預測結果:", predicted_labels)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 93s 10s/step - accuracy: 0.5216 - loss: 0.6932 
Epoch 2/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 79s 10s/step - accuracy: 0.4953 - loss: 0.6933 
Epoch 3/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 82s 10s/step - accuracy: 0.5077 - loss: 0.6932 
Epoch 4/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 84s 10s/step - accuracy: 0.4792 - loss: 0.6932 
Epoch 5/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 143s 11s/step - accuracy: 0.5024 - loss: 0.6932
Epoch 6/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 85s 11s/step - accuracy: 0.4863 - loss: 0.6932 
Epoch 7/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 90s 11s/step - accuracy: 0.4866 - loss: 0.6932 
Epoch 8/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 89s 11s/step - accuracy: 0.5156 - loss: 0.6931 
Epoch 9/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 91s 11s/step - accuracy: 0.5068 - loss: 0.6931 
Epoch 10/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 86s 11s/step - accuracy: 0.4987 - loss: 0.6932 
Epoch 11/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 82s 10s/step - accuracy: 0.5020 - loss: 0.6932 
Epoch 12/15
8/8 ━━━━━━━━━━━━━━━━━━━━ 90s 11s/step - accuracy: 0.4984 - los